In [ ]:
# synthetic_user_simulator.py
import random
import sys
import os
import logging

import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from typing import Dict, Tuple, List, Union

sys.path.append(os.path.abspath(os.path.join('..', '..', 'src', 'utils')))
import utils as ut

class SyntheticUserSimulator:
    """
    Esta clase simula usuarios que eligen vinos basados en preferencias sintéticas
    y evalúan si les gustó o no una recomendación.

    Objetivo: generar datasets sintéticos para entrenar los modelos de Machine Learning.
    """

    def __init__(self, wine_df: pd.DataFrame, meals_df: pd.DataFrame,
                 pairing_cols: List[str], taste_cols: List[str],
                 weights: Dict[str, float], top_n: int = 20):
        """
        Constructor: inicializa la clase con los datos base.

        Args:
        - wine_df: DataFrame de vinos.
        - meals_df: DataFrame de recetas/platos.
        - pairing_cols: Lista con nombre de las columnas a utilizar como pairing.
        - taste_cols: Lista con nombre de las columnas a utilizar como taste.
        - weights: Diccionario con pesos para cada componente del score (rating, price_quality, rating_qty, user_similarity, main_pairing).
        - top_n: Cantidad máxima de vinos que el usuario considera para elegir uno.
        """
        self.meals_df = meals_df.copy()
        self.pairing_cols = pairing_cols
        self.taste_cols = taste_cols
        self.weights = weights
        self.top_n = top_n
        self.scaler = MinMaxScaler()

        # Transformación del df para aplicar lógicas de usuario
        self.wine_df = wine_df.copy()
        self.tra_df = self.wine_df.copy()

        # Escalado de tastes
        self.taste_cols_scld = [taste + "_scld" for taste in self.taste_cols]
        self.tra_df[self.taste_cols_scld] = self.scaler.fit_transform(self.tra_df[self.taste_cols])

        # Perfiles de pairing (cuantiles y stds por pairing y taste)
        self._taste_profiles, self._taste_deviations = self._build_pairing_profile()

    def generate_user_profile(self) -> Dict:
        """
        Genera un perfil sintético de usuario (gustos, pairing, uvas, etc.)

        Returns:
        - Diccionario con los parámetros preferidos por el usuario.
        """
        pass  # <-- Acá vas a pegar tu función de generación de inputs aleatorios del usuario


    def _build_pairing_profile(self, quantiles=[0, .25, .5, .75, 1]) -> Tuple[Dict, Dict]:
        """
        Crea los perfiles de sabor por cuantiles por pairing.
        - Objetivo: que haga sentido el perfil elegido con el seleccionado por el user.

        Args:
        - quantiles: Lista con cuantiles a considerar.

        Returns:
        - profile: Diccionario con cuantiles por sabor y pairing.
        - deviations: Diccionario con desvíos estándar por sabor y pairing.
        """
        profiles = {}
        deviations = {}
        for pairing in self.pairing_cols:
            pairing_subset = self.tra_df[self.tra_df[pairing] == 1]
            profiles[pairing] = {}
            deviations[pairing] = {}
            for taste in self.taste_cols_scld:
                # Cuantiles
                taste_quantile = pairing_subset[taste].quantile(quantiles)
                profiles[pairing][taste] = taste_quantile

                # Desvío Estandar (para distribución normal en select_taste_profile)
                std = pairing_subset[taste].std()
                deviations[pairing][taste] = std

        return profiles, deviations
    

    def _select_random_meal(self) -> Dict[str, object]:
        """
        Selecciona una comida al azar y devuelve info útil para construir el perfil del usuario.

        Returns:
        - Dict con:
            - meal (pd.Series): la comida completa (ingredientes, columnas one-hot).
            - main_pairing (str): ingrediente principal en lowercase.
            - pairing_list (List[str]): lista de pairings que tienen valor 1.
        """
        i = np.random.randint(0, len(self.meals_df))
        selected_meal = self.meals_df.loc[i]

        main_pairing = selected_meal["Ingrediente Principal"].lower()

        # Excluye las 2 primeras columnas: "Comida" e "Ingrediente Principal"
        one_hot = selected_meal[2:]
        pairing_list = list(one_hot.index[one_hot > 0])

        return {
            "meal": selected_meal,
            "main_pairing": main_pairing,
            "pairing_list": pairing_list
        }
    
    
    def _select_taste_profile(self, main_pairing: str, prob: float = 0.2,
                          categories: List[str] = ["leve", "moderado", "marcado", "intenso"]) -> Dict[str, str]:
        """
        Selecciona un perfil de gustos de vino en base al pairing principal.
        Usa lógica probabilística con distribución normal y categorías definidas (sentido común + random).

        Returns:
        - Diccionario con el gusto preferido por cada variable de sabor (body, sweetness, etc.)
        """
        selected_profile = {}
        for taste in self.taste_cols_scld:
            quantiles = self._taste_profiles[main_pairing][taste]
            std = self._taste_deviations[main_pairing][taste]

            if random.random() < prob:
                # Opción aleatoria entre cuantiles medios
                quant_random_options = [
                    (quantiles.values[i] + quantiles.values[i+1])/2
                    for i in range(len(quantiles.values)-1)
                ]
                selected_val = np.random.choice(quant_random_options)
            else:
                median = quantiles[.5]
                min_val = quantiles[0]
                max_val = quantiles[1]
                selected_val = min(max(np.random.normal(loc=median, scale=std), min_val), max_val)
            
            selected_profile[taste] = self._get_category(selected_val, quantiles, categories)
        return selected_profile


    def _get_category(self, value, quantile_values: pd.Series,
                      categories: List[str] = ["leve", "moderado", "marcado", "intenso"]) -> Tuple[str, List[float]]:
        """
        Clasifica un valor en una categoría sensorial (leve, moderado, etc.) según cuantiles.

        Args:
        - value: valor a clasificar (escalado)
        - quantile_values: serie de cuantiles para esa variable sensorial
        - categories: etiquetas para cada rango de cuantiles

        Returns:
        - Tuple (categoría asignada, rango de valores del cuantil correspondiente)
        """
        if len(quantile_values)-1 != len(categories):
            raise IndexError("Length of quantiles ranges and categories does not match!")
        
        quant_values = quantile_values.values

        # Obtiene categoría y valor promedio del rango de los cuantiles
        for i in range(len(quant_values)-1):
            start_val = quant_values[i]
            end_val = quant_values[i+1]
            if start_val <= value < end_val:
                return categories[i], [start_val, end_val]
        
        # Gestiona valores atípicos (mayores que el máximo valor de los cuantiles)
        if value >= quant_values[-1]:
            return quant_values[-1], [quant_values[-2], quant_values[-1]]
        
        # Gestiona valores atípicos (menores que el mínimo valor de los cuantiles)
        if 0 <= value < quant_values[0]:
            return categories[0], [quant_values[0], quant_values[1]]
        
        # Error si el valor es inferior a cero
        raise ValueError(f"Passed value is less than zero! {value}")


    # Selección de uvas según probabilidad de uva en dataset
    def _select_grapes(self, grapes_df: pd.DataFrame,
                       top_n_grapes: int = 8,
                       min_grapes: int = 1,
                       max_grapes: int = 4) -> List[str]:
        """
        Selecciona aleatoriamente un set de uvas según su frecuencia en el dataset.

        Args:
        - grapes_df: dataframe con columnas de uvas (one-hot encoded).
        - top_n_grapes: cuántas uvas populares considerar (resto = Otras Uvas).
        - min_grapes: cantidad mínima de uvas a seleccionar.
        - max_grapes: cantidad máxima de uvas a seleccionar.

        Returns:
        - Lista de uvas seleccionadas.
        """
        top_grapes = self._get_top_grapes(grapes_df, top_n_grapes)
        total_grapes_in_df = sum(grapes_df.sum(axis=0))

        top_grapes_prob = top_grapes / total_grapes_in_df
        top_grapes_prob["Otras Uvas"] = 1 - sum(top_grapes_prob)
        grapes_prob = dict(top_grapes_prob)

        while True:
            grape_selection = [grape for grape, prob in grapes_prob.items() if random.random() < prob]
            if len(grape_selection) >= min_grapes:
                break
            
        return grape_selection[:max_grapes]
    

    def _get_top_grapes(self, grapes_df: pd.DataFrame, top_n_grapes: int) -> pd.Series:
        """
        Devuelve las top N uvas más frecuentes del dataset.

        Args:
        - grapes_df: dataframe con columnas de uvas (one-hot).
        - top_n_grapes: número de uvas más frecuentes a devolver.

        Returns:
        - Serie con el total de apariciones por uva (orden descendente).
        """
        top_grapes = grapes_df.sum(axis=0).sort_values(ascending=False).head(top_n_grapes)
        return top_grapes

    def select_price_range(
            self,
            df: pd.DataFrame,
            quantiles: List[float] = [.075, .125, .25, .375, .5, .625, .75, .875],
            low_prices_weights: List[float] = [.2, .3, .25, .15, .05, .025, .015, 0.01, 0, 0],
            min_dispersion: int = 2
    ) -> List[Union[int, None]]:
        """
        Selecciona un rango de precios realista con sesgo hacia precios bajos, simulando comportamiento de usuarios reales.

        Args:
        - df: dataframe que contiene la columna 'price'.
        - quantiles: cuantiles a considerar para definir posibles precios.
        - low_prices_weights: pesos para elegir precios bajos (más probabilidad al principio).
        - min_dispersion: distancia mínima entre el precio bajo y el alto en el rango de cuantiles.

        Returns:
        - List [min_price, max_price] donde max_price puede ser None si no hay límite superior.
        """
        # Eliminación de outliers (IQR * 1.5)
        no_outlier_price = ut.manage_outlier_IQR(df=df["price"], func="remove")
        price_quantiles = no_outlier_price.quantile(quantiles)

        # Adición de extremos (sin precio mínimo / máximo)
        price_quantiles[0], price_quantiles[1] = 0, -1
        price_quantiles = price_quantiles.sort_index()
        price_quantiles = pd.Series(price_quantiles.values)

        # Selección índice de cuantil de bajo precio
        low_price_index = random.choices(population=range(len(price_quantiles)), weights=low_prices_weights, k=1)[0]

        
        high_top_limit = len(price_quantiles)
        high_bottom_limit = low_price_index + min_dispersion if (low_price_index + min_dispersion) < len(price_quantiles) else len(price_quantiles)-1

        high_price_possible = range(high_bottom_limit, high_top_limit)

        high_prices_weights = [1 / x**1.5 for x in high_price_possible]

        high_price_index = random.choices(population=high_price_possible, weights=high_prices_weights, k=1)[0]

        price_range = [int(price_quantiles[low_price_index]), int(price_quantiles[high_price_index])]

        if price_range[1] == -1:
            price_range[1] = None

        return price_range


    def score_wines(self, user_profile: Dict) -> pd.DataFrame:
        """
        Asigna un score a cada vino según el perfil del usuario.

        Parameters:
        - user_profile: Diccionario con gustos y preferencias del usuario.

        Returns:
        - DataFrame con vinos y sus scores.
        """
        pass  # <-- Acá metés tu lógica de cálculo del score del vino

    def simulate_user_choice(self, scored_df: pd.DataFrame) -> pd.DataFrame:
        """
        Elige los top_n vinos con mayor score.

        Parameters:
        - scored_df: DataFrame con vinos y sus scores calculados

        Returns:
        - Subset del DataFrame con los top_n vinos seleccionados
        """
        pass  # <-- Vas a usar nlargest o similar para seleccionar vinos con mejores scores

    def simulate_like(self, wine: pd.Series, user_profile: Dict) -> int:
        """
        Determina si al usuario le gustó o no un vino en base al score y desviación.

        Parameters:
        - wine: Serie con info del vino.
        - user_profile: Diccionario del perfil del usuario.

        Returns:
        - 1 si le gustó, 0 si no.
        """
        pass  # <-- Tu lógica de simulate_like va acá

    def generate_synthetic_data(self, n_users: int) -> pd.DataFrame:
        """
        Genera datos sintéticos para n usuarios.

        Parameters:
        - n_users: Cantidad de usuarios sintéticos a simular.

        Returns:
        - DataFrame con columnas: user_id, wine_id, wine_score, liked
        """
        synthetic_data = []

        for i in range(n_users):
            user_profile = self.generate_user_profile()
            scored_wines = self.score_wines(user_profile)
            top_wines = self.simulate_user_choice(scored_wines)

            for _, row in top_wines.iterrows():
                liked = self.simulate_like(row, user_profile)
                synthetic_data.append({
                    'user_id': f'user_{i}',
                    'wine_id': row['wine_id'],
                    'wine_score': row['wine_score'],
                    'liked': liked
                })

        return pd.DataFrame(synthetic_data)

In [ ]:
# Class Main Inputs
wines_df = pd.read_csv("../../src/data/transformed/wines_clean.csv")
meals_df = pd.read_excel("../../src/data/raw/meals/Meals.xlsx")
pairings = pd.read_csv("../../src/data/processed/aux/pairings.csv")
pairings = list(pairings["pairings"])
taste_columns = ["body", "tannis", "sweetness", "acidity"]
weights = {}

# Grape Methods Input
grapes = pd.read_csv("../../src/data/processed/aux/grapes.csv")
grapes = grapes["grapes"]
grapes_df = wines_df[grapes]

user = SyntheticUserSimulator(
    wine_df=wines_df, meals_df=meals_df, pairing_cols=pairings, taste_cols=taste_columns, weights=weights)

profiles, deviations = user._build_pairing_profile()

meal = user._select_random_meal()

taste_profile = user._select_taste_profile(meal.get("main_pairing"))

grapes = user._select_grapes(grapes_df)


['Malbec']